In [1]:
'''
Learning about data processing using python, specifically using Pandas library
'''
import pandas as pd

In [17]:
''' Loading sqlite data using pandas '''
import sqlite3

conn= sqlite3.connect('data/db_backup_180719.sqlite3')

# get all tables
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())
print()

df_report = pd.read_sql_query('SELECT * FROM accounting_report', conn)
df_category = pd.read_sql_query('SELECT * FROM accounting_category', conn)

print(df_report.head())
print(df_category.head())

[('django_migrations',), ('sqlite_sequence',), ('auth_group',), ('auth_group_permissions',), ('auth_user_groups',), ('auth_user_user_permissions',), ('django_admin_log',), ('django_content_type',), ('auth_permission',), ('auth_user',), ('django_session',), ('accounting_category',), ('profiles_wallet',), ('accounting_report',), ('profiles_activationkey',)]

   id  is_expense  amount  category_id  owner_id        date
0   4           0  100000            3         1  2019-02-12
1   5           0   10000            4         1  2019-02-17
2   6           0  100000            5         1  2019-02-19
3   7           0   11000            4         1  2019-02-20
4   8           0   29000            4         1  2019-02-22
   id          name
0   1        snacks
1   2         lunch
2   3      paycheck
3   4  pocket money
4   5  from savings


In [45]:
# show only selected column
print(df[['category_id', 'amount']].head())
print()

# make new df using existing df with selected column
df_details = df[['is_expense', 'amount', 'category_id', 'date']]
print(df_details.head())
print()

# select row using value
a_row = df_category.loc[df_category['id'] == 10]

# print values
print(a_row['name'].values)
print()

# make new columns which rows defined based by existing rows-column
df_details['category'] = df_details['category_id'].map(lambda x: df_category.loc[df_category['id'] == x]['name'].values[0])
print(df_details[['category']].head())
print()

df_details['status'] = df_details['is_expense'].map(lambda x: "Expense" if x==1 else "Income")
print(df_details[['status']].head())
print()

# drop column
df_details = df_details.drop(['category_id', 'is_expense'], axis=1)
print(df_details.head())

   category_id  amount
0            3  100000
1            4   10000
2            5  100000
3            4   11000
4            4   29000

   is_expense  amount  category_id        date
0           0  100000            3  2019-02-12
1           0   10000            4  2019-02-17
2           0  100000            5  2019-02-19
3           0   11000            4  2019-02-20
4           0   29000            4  2019-02-22

['laundry']

       category
0      paycheck
1  pocket money
2  from savings
3  pocket money
4  pocket money

   status
0  Income
1  Income
2  Income
3  Income
4  Income

   amount        date      category  status
0  100000  2019-02-12      paycheck  Income
1   10000  2019-02-17  pocket money  Income
2  100000  2019-02-19  from savings  Income
3   11000  2019-02-20  pocket money  Income
4   29000  2019-02-22  pocket money  Income


/home/rmyrnvn/Vault/DataScience/venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [100]:
import datetime

# sort dataframe by date ascending
df_details['date'] = pd.to_datetime(df_details['date'])
df_details = df_details.sort_values(by=['date'])
print(df_details.head())
print()

# show rows with specific values
category = "snacks"
cat = df_details[df_details.category == category]
# cat = df_details[df_details.status == "Expense"]
print(cat.head())
print()

# some calculation
total_cat = sum(cat[['amount']].values)[0]
duration = (cat['date'].values[-1] - cat['date'].values[0])
days = duration.astype('timedelta64[D]').astype(int) + 1

print(f"Duration : {days} days")
print(f"Total {category} Expenses : Rp {total_cat:,}")
print(f"{category} expenses per day : Rp {round(total_cat/days):,}")

    amount       date  category   status
0   100000 2019-02-12  paycheck   Income
21    8000 2019-02-13  thai tea  Expense
20    1500 2019-02-13    snacks  Expense
19   13500 2019-02-13   laundry  Expense
24    2000 2019-02-15    snacks  Expense

    amount       date category   status
20    1500 2019-02-13   snacks  Expense
24    2000 2019-02-15   snacks  Expense
23    5000 2019-02-15   snacks  Expense
28    1000 2019-02-16   snacks  Expense
27   11000 2019-02-16   snacks  Expense

Duration : 155 days
Total snacks Expenses : Rp 677,500
snacks expenses per day : Rp 4,371.0
